In [1]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [2]:
tf.config.set_soft_device_placement(True)
# tf.debugging.set_log_device_placement(True)
dataset = pd.read_csv("data/train.csv", header=None)
# dataset = tf.keras.utils.get_file('train', 'data/train.csv',
#                                   untar=True, cache_dir='.',
#                                   cache_subdir='')

# dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')

# train_dir = os.path.join('data/test.csv', 'train')


In [3]:
train_ds = np.array(dataset.iloc[:, 2])
train_ds = np.array([np.array([x]) for x in train_ds])


In [4]:
val_ds = np.array(dataset.iloc[:, 0])
val_ds = np.array([np.array(x-1)  for x in val_ds])


In [5]:
with tf.device('/CPU:0'):
  AUTOTUNE = tf.data.AUTOTUNE
  batch_size = 32
  seed = 42
  tfhub_handle_preprocess = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2'
  tfhub_handle_encoder = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1'
  bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)
  bert_model = hub.KerasLayer(tfhub_handle_encoder)

2022-04-15 14:49:46.804092: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-15 14:49:47.124482: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9806 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060, pci bus id: 0000:41:00.0, compute capability: 8.6


In [6]:
text_test = ['this is such an amazing movie!']
text_preprocessed = bert_preprocess_model(text_test)

In [6]:
def build_classifier_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  net = tf.keras.layers.Dense(5, activation=tf.sigmoid, name='classifier')(net)
  return tf.keras.Model(text_input, net)

In [7]:
classifier_model = build_classifier_model()

In [8]:
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = tf.keras.metrics.SparseCategoricalAccuracy()

In [9]:
epochs = 5
# steps_per_epoch = tf.data.experimental.cardinality(train_ds).numpy()
# num_train_steps = steps_per_epoch * epochs
# num_warmup_steps = int(0.1*num_train_steps)

# init_lr = 3e-5
# optimizer = optimization.create_optimizer(init_lr=init_lr,
#                                           num_train_steps=num_train_steps,
#                                           num_warmup_steps=num_warmup_steps,
#                                           optimizer_type='adamw')

In [10]:
classifier_model.compile(optimizer='adam',
                         loss=loss,
                         metrics=metrics)

In [12]:
history = classifier_model.fit(x=train_ds,
                               y=val_ds,
                               epochs=epochs)

Epoch 1/5
 4590/93750 [>.............................] - ETA: 3:53:11 - loss: 1.6095 - sparse_categorical_accuracy: 0.1981

KeyboardInterrupt: 

In [3]:
gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]